<a href="https://colab.research.google.com/github/skbhere/Foot-or-not-CNN-Binary-classifier-/blob/master/Transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam

In [2]:
mobile = keras.applications.mobilenet.MobileNet()
def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

17227776/17225924 [==============================] - 0s 0us/step


In [4]:
preprocessed_image = prepare_image('Lighthouse.jpg')
predictions = mobile.predict(preprocessed_image)
results = imagenet_utils.decode_predictions(predictions)
results

[[('n02814860', 'beacon', 0.83203715),
  ('n02980441', 'castle', 0.1304102),
  ('n03781244', 'monastery', 0.012002675),
  ('n03028079', 'church', 0.00917956),
  ('n09399592', 'promontory', 0.0034800267)]]

In [5]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(2,activation='softmax')(x) #final layer with softmax activation

17227776/17225924 [==============================] - 0s 0us/step


In [6]:
model=Model(inputs=base_model.input,outputs=preds)

In [13]:
# for i,layer in enumerate(model.layers):
#   # print(i,layer.name)

In [14]:
for layer in model.layers:
    layer.trainable=False
# or if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('/content/drive/My Drive/data/foot or not/',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 293 images belonging to 2 classes.


In [17]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=10)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
9/9 [==============================] - 53s 6s/step - loss: 0.8311 - accuracy: 0.8161
Epoch 2/10
9/9 [==============================] - 31s 3s/step - loss: 0.0925 - accuracy: 0.9808
Epoch 3/10
9/9 [==============================] - 32s 4s/step - loss: 0.0339 - accuracy: 0.9885
Epoch 4/10
9/9 [==============================] - 31s 3s/step - loss: 0.0303 - accuracy: 0.9847
Epoch 5/10
9/9 [==============================] - 30s 3s/step - loss: 0.0551 - accuracy: 0.9885
Epoch 6/10
9/9 [==============================] - 30s 3s/step - loss: 0.0672 - accuracy: 0.9847
Epoch 7/10
9/9 [==============================] - 35s 4s/step - loss: 0.0500 - accuracy: 0.9861
Epoch 8/10
9/9 [==============================] - 30s 3s/step - loss: 0.0495 - accuracy: 0.9732
Epoch 9/10
9/9 [==============================] - 31s 3s/step - loss: 0.1845 - accuracy: 0.9349
Epoch 10/10
9/9 [==============================] - 32s 4s/st

In [30]:
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(150, 150))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor
  
#img_path = 'C:/Users/Ferhat/Python Code/Workshop/Tensoorflow transfer learning/blue_tit.jpg'
img_path = '/content/122345.jpg'
new_image = load_image(img_path)

pred = model.predict(new_image)

pred

array([[1.0000000e+00, 3.0799998e-11]], dtype=float32)